In [ ]:
import cv2
import numpy as np
import time
from jetbot import Camera, Robot
from IPython.display import display
import ipywidgets as widgets
from PID import PID
from jetbot.utils import bgr8_to_jpeg

# -------------------------
# Parameters
# -------------------------
camera_width = 320
camera_height = 240

forward_speed = 0.18
roi_height_fraction = 0.4  # bottom 40% of frame
roi_width_fraction = 1/3   # middle third horizontally

pid = PID(Kp=0.001, Ki=0.0, Kd=0.0005, setpoint=camera_width / 2)

# -------------------------
# Hardware
# -------------------------
camera = Camera.instance(width=camera_width, height=camera_height)
robot = Robot()

# -------------------------
# UI Controls
# -------------------------
image_widget = widgets.Image(format='jpeg', width=camera_width, height=camera_height)
start_button = widgets.Button(description='START', button_style='success', layout=widgets.Layout(width='100px', height='50px'))
stop_button = widgets.Button(description='STOP', button_style='danger', layout=widgets.Layout(width='100px', height='50px'))

robot_stop = True

def start_robot(b):
    global robot_stop
    robot_stop = False

def stop_robot(b):
    global robot_stop
    robot_stop = True
    robot.stop()

start_button.on_click(start_robot)
stop_button.on_click(stop_robot)

display(widgets.VBox([image_widget, widgets.HBox([start_button, stop_button])]))

# -------------------------
# Line Following Function
# -------------------------
def follow_black_line(frame):
    global robot_stop

    h, w, _ = frame.shape

    # Extract bottom-middle ROI
    y1 = int(h * (1 - roi_height_fraction))
    y2 = h
    x1 = int(w * (1 - roi_width_fraction) / 2)
    x2 = int(w * (1 + roi_width_fraction) / 2)
    roi = frame[y1:y2, x1:x2]

    # Convert to grayscale and threshold
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    mask = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 15, 5
    )

    # Clean up noise
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find line centroid
    M = cv2.moments(mask)
    line_found = M["m00"] > 0

    if line_found:
        cx = int(M["m10"] / M["m00"]) + x1  # adjust for ROI offset
        cv2.circle(frame, (cx, h - 10), 5, (0,0,255), -1)
        steering = pid.update(cx)
        left_speed = forward_speed + steering
        right_speed = forward_speed - steering
    else:
        left_speed = right_speed = 0

    # Stop if pressed
    if robot_stop:
        left_speed = right_speed = 0

    robot.set_motors(left_speed, right_speed)

    # Debug visualization
    vis = frame.copy()
    cv2.rectangle(vis, (x1, y1), (x2, y2), (255, 0, 0), 2)
    image_widget.value = bgr8_to_jpeg(vis)

# -------------------------
# Attach camera callback
# -------------------------
camera.observe(lambda change: follow_black_line(change['new']), names='value')

In [ ]:
robot.stop()
camera.stop()